In [ ]:
# Standard library imports
import time
from pathlib import Path

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


# Custom package import
# from cifar10_tools.pytorch import train_model

# Set random seed for reproducibility
torch.manual_seed(315)
np.random.seed(315)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [24]:
import os
os.listdir("../data") # Asks the OS: “What files/folders exist directly inside data/?”


['Face_mask_detection', 'Face_mask_detection.zip']

In [23]:
import zipfile
from pathlib import Path
import os

zip_path = Path("../data/Face_mask_detection.zip")
extract_dir = Path("../data/Face_mask_detection")

extract_dir.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

print("Extracted files:", os.listdir(extract_dir))

Extracted files: ['data']


In [ ]:
# Hyperparameters
bacth_size = 50
learning_rate = 1e-3
epoch = 30
print_every = 5



In [ ]:
#Data Loader 



transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = ImageFolder(
    root="../data/Face_mask_detection",
    transform=transform
)


In [ ]:
dataset

AttributeError: 'ImageFolder' object has no attribute 'head'